In [ ]:
!pip install rank_bm25

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import pandas as pd
titles_path = './utils/titles_corrected'
titles_df = pd.read_csv(titles_path)
titles_df.head()

In [ ]:
import pandas as pd
queries_path = './utils/queries_corrected'
queries_df = pd.read_csv(queries_path)
queries_df.head()

In [ ]:
from rank_bm25 import BM25Okapi, BM25Plus, BM25L

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams(text, n=1, word=True):
    if word:
        n_grams = ngrams(word_tokenize(text), n)
        return [ ' '.join(grams) for grams in n_grams]
    else:
        n_grams = ngrams(text, n)
        return [ ''.join(grams) for grams in n_grams]

In [ ]:
import pandas as pd
sample_subm = pd.read_csv('./data/ranking-long-tail-queries-fall-2020/sample.csv/sample.csv')
tuples = [tuple(sample_subm.values[i]) for i in range(len(sample_subm))]

groups_mapping = dict()
for elem in tuples:
    if elem[0] not in groups_mapping:
        groups_mapping[elem[0]] = [elem[1]]
    else:
        groups_mapping[elem[0]].append(elem[1])

In [ ]:
import numpy as np
queries_in_mapping = np.array(list(groups_mapping.keys()))
queries_in_mapping

In [ ]:
queries = queries_df.values[queries_in_mapping, :]
queries

In [ ]:
queries_arr = list(map(str, queries[:, 1]))

### Add features

In [ ]:
import pandas as pd
marks_path = './data/ranking-long-tail-queries-fall-2020/train.marks.tsv/train.marks.tsv'
marks_df = pd.read_csv(marks_path, '\t', header=None)

In [ ]:
tuples = [tuple(marks_df.values[i]) for i in range(len(marks_df))]

train_groups_mapping = dict()
for elem in tuples:
    if elem[0] not in train_groups_mapping:
        train_groups_mapping[elem[0]] = [elem[1]]
    else:
        train_groups_mapping[elem[0]].append(elem[1])

In [ ]:
import numpy as np
train_queries_in_mapping = np.array(list(train_groups_mapping.keys()))
train_queries_in_mapping

In [ ]:
titles_df.values[:, 1][0]

In [ ]:
get_ngrams(titles_df.values[:, 1][0], n=1)

In [ ]:
from tqdm import notebook
all_sents = []
for title in notebook.tqdm(titles_df.values[:, 1]):
    current_sent = get_ngrams(str(title), n=1)
    all_sents.append(current_sent)

In [ ]:
bm25 = BM25Plus(all_sents)

DocsId = []
QueryId = []

queries_tr = queries_df.values[train_queries_in_mapping, :]
queries_arr_tr = list(map(str, queries_tr[:, 1]))

bm25_scores = []
for q_id, query in notebook.tqdm(zip(train_queries_in_mapping, queries_arr_tr)):

    batch_scores = bm25.get_batch_scores(get_ngrams(query, n=1), train_groups_mapping[q_id])
  
    for doc_id, score in zip(train_groups_mapping[q_id], batch_scores):
        QueryId.append(q_id)
        DocsId.append(doc_id)
        bm25_scores.append(score)
        

queries_test = queries_df.values[queries_in_mapping, :]
queries_arr_test = list(map(str, queries_test[:, 1]))

for q_id, query in notebook.tqdm(zip(queries_in_mapping, queries_arr_test)):

    batch_scores = bm25.get_batch_scores(get_ngrams(query, n=1), groups_mapping[q_id])

    for doc_id, score in zip(groups_mapping[q_id], batch_scores):
        QueryId.append(q_id)
        DocsId.append(doc_id)
        bm25_scores.append(score)

In [ ]:
col_name = 'Bm25Plus1word_ru'
df = pd.DataFrame({
    'QueryId' : QueryId,
    'DocumentId' : DocsId,
    col_name: bm25_scores
})

In [ ]:
word_gram = 1
bm25 = BM25L(all_sents)

bm25_scores = []
for q_id, query in notebook.tqdm(zip(train_queries_in_mapping, queries_arr_tr)):

    batch_scores = bm25.get_batch_scores(get_ngrams(query, n=word_gram), train_groups_mapping[q_id])
    
    for doc_id, score in zip(train_groups_mapping[q_id], batch_scores):
        bm25_scores.append(score)

for q_id, query in notebook.tqdm(zip(queries_in_mapping, queries_arr_test)):

    batch_scores = bm25.get_batch_scores(get_ngrams(query, n=word_gram), groups_mapping[q_id])

    for doc_id, score in zip(groups_mapping[q_id], batch_scores):
        bm25_scores.append(score)
df['Bm25OL1word_ru'] = bm25_scores

In [ ]:
bm25 = BM25Okapi(all_sents)

bm25_scores = []
for q_id, query in notebook.tqdm(zip(train_queries_in_mapping, queries_arr_tr)):

    batch_scores = bm25.get_batch_scores(get_ngrams(query, n=word_gram), train_groups_mapping[q_id])
    
    for doc_id, score in zip(train_groups_mapping[q_id], batch_scores):
        bm25_scores.append(score)

for q_id, query in notebook.tqdm(zip(queries_in_mapping, queries_arr_test)):

    batch_scores = bm25.get_batch_scores(get_ngrams(query, n=word_gram), groups_mapping[q_id])

    for doc_id, score in zip(groups_mapping[q_id], batch_scores):
        bm25_scores.append(score)
df['Bm25Okapi1word_ru'] = bm25_scores

In [ ]:
path_to_parsed = './utils/parsed/'
import os
parsed_filenames = sorted(os.listdir(path_to_parsed), key=int)

all_sents_text = []
for docname in notebook.tqdm(parsed_filenames):
    with open(path_to_parsed + docname, 'r') as f:
        current_sent = get_ngrams(str(f.read()), n=1)
        all_sents_text.append(current_sent)

In [ ]:
bm25 = BM25Plus(all_sents_text)

DocsId = []
QueryId = []

queries_tr = queries_df.values[train_queries_in_mapping, :]
queries_arr_tr = list(map(str, queries_tr[:, 1]))

bm25_scores = []
for q_id, query in notebook.tqdm(zip(train_queries_in_mapping, queries_arr_tr)):

    batch_scores = bm25.get_batch_scores(get_ngrams(query, n=1), train_groups_mapping[q_id])
  
    for doc_id, score in zip(train_groups_mapping[q_id], batch_scores):
        QueryId.append(q_id)
        DocsId.append(doc_id)
        bm25_scores.append(score)
        

queries_test = queries_df.values[queries_in_mapping, :]
queries_arr_test = list(map(str, queries_test[:, 1]))

for q_id, query in notebook.tqdm(zip(queries_in_mapping, queries_arr_test)):

    batch_scores = bm25.get_batch_scores(get_ngrams(query, n=1), groups_mapping[q_id])

    for doc_id, score in zip(groups_mapping[q_id], batch_scores):
        QueryId.append(q_id)
        DocsId.append(doc_id)
        bm25_scores.append(score)
df['Bm25Texts'] = bm25_scores

In [ ]:
df.to_csv('lex', index=None)